In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [91]:
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [3]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [5]:
print(train.head())

In [52]:
def plotimgi(i):
    img0 = train.iloc[i][1:].to_numpy()
    img0 = np.reshape(img0, (28,28))
    plt.imshow(img0)
    return False
plotimgi(3)
train['label'][3]

In [141]:
X = train.drop(['label'], axis = 1) > 50
y = train['label']

#one hot encoding
y = to_categorical(y)

In [142]:

#X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = .3, random_state = 42)
#defining the architecture
model = Sequential(
[Dense(100,input_shape = (X_train.shape[1],), activation = 'relu'),
 Dense(100, activation = 'relu'),
 Dense(100, activation = 'relu'),
 Dense(10, activation = 'softmax')
])

#early stop so it doesn't get stuck for too long
early_stop = EarlyStopping(patience = 4)

# compiling with adam and fitting
model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy')
model.fit(X,y, epochs = 30, validation_split = .3, callbacks = early_stop)

In [127]:
#submitting
sub = pd.read_csv('../input/digit-recognizer/sample_submission.csv')
pred = model.predict(test)
sub['Label'] = np.argmax(pred, axis = -1)

sub.to_csv('submission.csv', index = False)

In [134]:
# Testing against a nontuned random forest classifier

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(oob_score = True)
rf.fit(X,y)


In [135]:
rf.oob_score_